In [ ]:
#1
# объявление основных переменных/функций


import os
import os.path
import shutil
import cgitb, sys
from google.colab import output as cou
from google.colab import drive
import re
from re import search
import subprocess
from subprocess import getoutput
import threading
from IPython.display import HTML, Javascript, display, clear_output
from ipython_genutils.py3compat import builtin_mod
import ipywidgets as widgets
import requests
import glob
import time
from datetime import timedelta
import distutils.dir_util
from IPython.utils import capture
import multiprocessing
import random
import string
from tqdm import tqdm
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor, as_completed
import pickle
import platform


cfg = "/content/.config/"
#переменная !$dl для сокращения команды загрузки, перед использованием обязательно задать переменную пути $path
path = '/content/'
wget = "wget -nv -t 10 --show-progress --progress=bar -q --content-disposition "
dl = f"{wget} -P {path} "
wq = "wget -q -nv -t 10 --content-disposition"
lz4 = "tar -xI lz4 -f"
dvach = "https://huggingface.co/2ch/models/resolve/main/"
sf = "https://huggingface.co/utnah/safetensors/resolve/main/"
pt = "https://huggingface.co/utnah/ckpt/resolve/main/"
vae = "https://huggingface.co/utnah/vae/resolve/main/"
esrgan = "https://huggingface.co/utnah/esrgan/resolve/main/"
lora = "https://huggingface.co/utnah/lora/resolve/main/"
hyper = "https://huggingface.co/utnah/hypernetworks/resolve/main/"
dl_deps = "wget -q -t 10 --content-disposition https://huggingface.co/utnah/deps/resolve/main/"
deps = "https://huggingface.co/utnah/deps/resolve/main/"
pip = f"/usr/local/lib/python{'.'.join(platform.python_version().split('.')[:2])}/dist-packages/"
gh = "https://github.com/PR0LAPSE/StableDiffusionWebUIColab/raw/main/src/"
dl_gh = f"wget -q -t 10 --content-disposition  {gh}"
dl_gh_alt = f"wget -q -t 10 --content-disposition https://github.com/xxxOVALxxx/colab/raw/main/"
end = '/content/.config/end'
deps_bar = f"{gh}depsprogressbar" # прогрессбар во время распаковки пакетов: loader(deps)
inprogress = f"{gh}loader_anim" # заглушка на css во время выполнения чего-либо: loader(inprogress)

m = HTML(requests.get(gh+'st').text)
ok = ""
with capture.capture_output() as stored_variables:
  %store cfg
  %store wget
  %store sf
  %store pt
  %store esrgan
  %store lora
  %store hyper
  %store dl_deps
  %store deps
  %store gh
  %store dl_gh
  %store end
  %store deps_bar
  %store inprogress
  %store m

vars_data = (cfg, wget, dl, wq, lz4, sf, pt, vae, esrgan, hyper, dl_deps, deps, pip, gh, dl_gh, end, deps_bar, inprogress, m)  # все переменные для сохранения в файл
with open(f'{cfg}variables', 'wb') as vars:
    pickle.dump(vars_data, vars)

def loader(html): # для подгрузки html/svg
  display(HTML(html, metadata=dict(isolated=True)))

def run_nb(filename):
  attempt = 0
  while not os.path.isfile(f"/content/.config/{filename}.ipynb") and attempt < 10:
    !wget -q https://github.com/xxxOVALxxx/colab/raw/main/{filename}.ipynb -P /content/.config
    attempt += 1
  %run /content/.config/{filename}.ipynb
  !rm /content/.config/{filename}.ipynb  > /dev/null 2>&1

#простая качалка (для загрузки архива с зависимостями)
def download(url: str, fname: str, desc: str, chunk_size=1024):
  try:
    resp = requests.get(url, stream=True)
    total = int(resp.headers.get('content-length', 0))
    with open(fname, 'wb') as file, tqdm(
      desc=desc,
      total=total,
      unit='iB',
      unit_scale=True,
      unit_divisor=1024,
    ) as bar:
        for data in resp.iter_content(chunk_size=chunk_size):
          size = file.write(data)
          bar.update(size)
  except KeyboardInterrupt:
    print("\nзагрузка отменена")
  except Exception as error:
    print(f"загрузка не удалась:\n{error}")

def ng():
  end_attempt = 0
  cfg_attempt = 0
  while not os.path.isfile("/content/.config/end") and end_attempt < 10:
    !$dl_gh'end' -O /content/.config/end # svg-баннер после завершения
    end_attempt += 1
  while not os.path.isfile("/content/.config/cfg") and cfg_attempt < 10:
    !$dl_gh'zrpmdNELTAkW' -O /content/.config/cfg # нгрок-токены
    cfg_attempt += 1

#распаковка зависимостей (которые ставятся через pip)
def depinst_old():
  #download(f"{deps}d", "/content/d", "зависимости")
  #!pip uninstall -y torch torchvision xformers triton torchtext
  clear()
  loader(deps_bar)
  !$wq $deps'linux/linux' -P $cfg'linux'
  !cat $cfg'linux/linux' | xargs -P 100 -n1 $wq -P $cfg'linux'
  !sudo dpkg --force-all -i {cfg}linux/*.deb > /dev/null 2>&1
  !$lz4 {cfg}linux/hf_clip.tar.lz4 --directory=/ 
  !rm -rf $cfg'linux'
  with open(os.path.join(cfg, "deps"), "w") as dep:
    dep.write("\n".join([f"{deps}d/deps_{{:03d}}".format(i) for i in range(1, 48)]))
  !rm -rf /dev/shm/*
  !mount -o remount,size=11G,exec /dev/shm
  !cat {cfg}deps | xargs -P 100 -n1 $wq -P /dev/shm/
  !cat /dev/shm/deps_* > /dev/shm/deps.tar.lz4
  pip_trails = ["torchvision.libs", "xformers", "triton", "torchtext", "torchvision.libs", "torchvision", "torchaudio"]
  for pip_trail in pip_trails:
    pip_path = os.path.join(pip, pip_trail)
    if os.path.exists(pip_path):
      os.system(f"sudo rm -rf {pip_path}")
  dist_trails = re.compile(r'(xformers|triton|torch(text|vision|vision\.libs|audio|))-\S+.dist-info')
  for item in os.listdir(pip):
    if dist_trails.match(item):
      dist_path = os.path.join(pip, item)
      os.system(f"sudo rm -rf {dist_path}")
  !$lz4 /dev/shm/deps.tar.lz4 --overwrite-dir -C $pip
  !rm -rf /dev/shm/* {cfg}deps
  %env LD_PRELOAD=/lib/libtcmalloc_minimal.so.4
  %env PYTORCH_CUDA_ALLOC_CONF=garbage_collection_threshold:0.9,max_split_size_mb:512
  %env TF_CPP_MIN_LOG_LEVEL=3
  clear()

def depinst():
  download(f"{deps}deps.7z", "/content/deps.7z", "зависимости")
  clear()
  loader(deps_bar)
  !7z -bso0 -bd -mmt4 -slp -y x /content/deps.7z -o/usr/local/lib/python3.10/dist-packages > /dev/null 2>&1
  !rm /content/deps.7z
  %env LD_PRELOAD=/lib/libtcmalloc_minimal.so.4
  %env PYTORCH_CUDA_ALLOC_CONF=garbage_collection_threshold:0.9,max_split_size_mb:512
  %env TF_CPP_MIN_LOG_LEVEL=3
  clear()

# определение папки A1111
def generate_random_file_names():
  return (name:=(''.join(random.choices(string.ascii_lowercase, k=60)))) and (fname:=name[:20], wname:=name[20:40]+'.py', lname:=name[40:]+'.py', f"/content/{fname}/")

def find_dir():
  root = "/content"
  for root, dirs, files in os.walk(root):
    for dir_name in dirs:
      if len(dir_name) == 20 and 'huge_rectal_prolapse_in_google_colab' in os.listdir(os.path.join(root, dir_name)):
        fpath = os.path.join(root, dir_name)+"/"
        return fpath

def flara():
  import os, re
  from subprocess import getoutput
  extra_urls = '/content/extra_urls'
  if os.path.exists(extra_urls):
    os.remove(extra_urls)
  if not os.path.exists('/tools/node/bin/lt'):
    !npm install -g localtunnel > /dev/null 2>&1
  if not os.path.exists('/usr/bin/bore'):
    !curl -Ls https://github.com/ekzhang/bore/releases/download/v0.4.0/bore-v0.4.0-x86_64-unknown-linux-musl.tar.gz | tar zx -C /usr/bin > /dev/null 2>&1
  if not os.path.exists('/usr/bin/cloudflared'):
    !curl -Lo /usr/bin/cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 && chmod +x /usr/bin/cloudflared > /dev/null 2>&1
  !nohup lt --port 7860 >> $extra_urls 2>&1 &
  !echo >> $extra_urls
  !nohup bore local 7860 --to bore.pub >> $extra_urls 2>&1 &
  !echo >> $extra_urls
  clf_attempt = 0
  while clf_attempt < 5:
    !nohup cloudflared tunnel --url localhost:7860 > /content/clf 2>&1 &
    clf_attempt += 1
    !sleep 2
    if "has been created" in getoutput(f'cat /content/clf'):
      with open("/content/clf", 'r') as file, open("/content/clfr", 'w') as output_file:
        for line in file:
          match = re.search(r'https://(\S+\.trycloudflare\.com)', line)
          if match:
            output_file.write(f'\033[01;38;05;112m⯈\033[0m Ссылка на Клаудфлару: https://{match.group(1)}\n')
            os.remove('/content/clf')
      break
    else:
      os.remove('/content/clf')
  links = []
  with open(extra_urls, 'r') as file:
    for line in file:
      match = re.search(r'your\s+url\s+is:\s+(https://\S+)', line)
      if match:
        links.append(f'\033[01;38;05;112m⯈\033[0m Ссылка на Локалт: {match.group(1)}\n')
      match = re.search(r'\s+listening\s+at\s+bore\.pub:(\d+)', line)
      if match:
        links.append(f'\033[01;38;05;112m⯈\033[0m Ссылка на Бор: http://bore.pub:{match.group(1)}\n')
  with open('/content/lb', 'w') as file:
    for link in links: file.write(link)
  if os.path.exists('/content/clfr'):
    with open('/content/clfr')as f1,open('/content/lb','a')as f2:
      for line in f1: f2.write(line)
    os.remove('/content/clfr')
    os.remove(extra_urls)
    os.rename('/content/lb', extra_urls)
  else:
    os.remove(extra_urls)
    os.rename('/content/lb', extra_urls)
  print(getoutput('cat /content/extra_urls'))
  #try:
  #  from google.colab.output import serve_kernel_port_as_window
  #  from google.colab.output import eval_js
  #  print("\033[01;38;05;112m⯈\033[0m Ссылка на Гугл:", eval_js( "google.colab.kernel.proxyPort(" + str(7860) + ")" ))
  #except:
  #  None

# css-файл для внедрения в виджеты
css_setup_widget = requests.get(f"{gh}css_setup_widget.html").text
def setup_css():
  display(HTML(f'''
    {css_setup_widget}
'''))

# функция воспроизведения аудио
def aloud(file):
  try:
    from google.colab import output as cou
    if file.startswith("http"):
      cou.eval_js('new Audio(file).play()')
    else:
      cou.eval_js(f'new Audio("{gh}{file}").play()')
  except Exception as e:
    print(f"{e}: твой браузер не поддерживает автоматическое воспроизведение аудио")


#проигрывание аудио, чтобы вкладка не убивалась
def mobile_player():
  try:
    cou.eval_js("var audio = new Audio('https://github.com/PR0LAPSE/StableDiffusionWebUIColab/raw/main/src/psyattack.wav'); audio.addEventListener('ended', function() { audio.currentTime = 0; audio.play(); }, false); audio.play();")
  except Exception as e:
    print(f"{e}: твой браузер не поддерживает автоматическое воспроизведение аудио")

# установка времени запуска колаба для таймера
try:
  launching
except:
  launching = int(time.time())-6
  with capture.capture_output() as stored_variables:
    %store launching

# автоматическое открытие ссылок градио и нгрок
def auto_opener():
  display(HTML('''
  <script>
  var linkOpened = false;
  function checkPage() {
    if (linkOpened) {
      return;
    }
    var links = document.getElementsByTagName("a");
    for (var i = 0; i < links.length; i++) {
      if (links[i].href.indexOf("gradio") !== -1) {
        window.open(links[i].href, '_blank');
        var ngroklink = document.querySelector('[href*="ngrok"]');
        window.open(ngroklink, '_blank');
        linkOpened = true;
        return;
      }
    }
  }
  checkPage();
  var observer = new MutationObserver(function(mutations) {
    mutations.forEach(function(mutation) {
      checkPage();
    });
  });
  observer.observe(document.body, { childList: true, subtree: true });
  </script>
  '''))


